# HOMES和歌山県戸建て賃貸、複数ページを取得

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
 
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
header = {
    'User-Agent': user_agent
}

# Last Pageの検出
first_page_url ='https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=1'
first_res = requests.get(first_page_url, headers=header)
soup_first = BeautifulSoup(first_res.text, 'html.parser')
last_page = soup_first.find('li', class_='lastPage').text
final_page = int(last_page)+1

url = 'https://www.homes.co.jp/chintai/kodate/wakayama/list/?page={}'

# 変数d_listに空のリストを作成する
d_list=[]

for i in range(1,final_page):
    print(len(d_list))

    # 変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)
    
    # print()してURLを確認する
    print(target_url)

    # Requestを使ってURLにアクセスする
    res = requests.get(target_url, headers=header)

    # 1秒間のスリープを挟む
    sleep(1)
    
    # 取得したHTMLをBeautiful Soupで解析する
    soup = BeautifulSoup(res.text, 'html.parser')

    bukken_units = soup.find_all('div', class_='mod-mergeBuilding--rent--photo rKodate ui-frame ui-frame-cacao-bar')

    for bukken_unit in bukken_units:

        # specとsummaryの表を抽出
        bukken_spec = bukken_unit.find('div', class_='bukkenSpec')
        bukken_summary = bukken_unit.find('table', class_='unitSummary unitResidenceSummary')

        # 物件名の抽出
        name = bukken_unit.find('span', 'bukkenName prg-detailLinkTrigger').text

        # 所在地、交通、築年数、階数をbukken_specから抽出
        place = bukken_spec.find_all('td')[0].text
        stations = bukken_spec.find_all('span', class_='prg-stationText')
        station1 = stations[0].text if len(stations) > 0 else None
        station2 = stations[1].text if len(stations) > 1 else None
        age_data = bukken_spec.find_all('td')[2].text.split(' / ')
        age = age_data[0]
        floor = age_data[1]

        # 家賃、間取り、面積、特徴をbukken_summaryから抽出
        rent = bukken_summary.find('span', class_='priceLabel').text
        madori = bukken_summary.find('td', class_='layout').contents[0].strip()
        menseki = bukken_summary.find('td', class_='layout').contents[2].strip()

        # featureはNoneがあるため考慮する
        feature_element = bukken_unit.find(class_="textFeatureComment")
        feature = feature_element.text if feature_element is not None else None

        # 辞書の作成
        d = {
            'name':name,
            'place':place,
            'station1':station1,
            'station2':station2,
            'age':age,
            'floor':floor,
            'rent':rent,
            'madori':madori,
            'menseki':menseki,
            'feature':feature,    
        }

        # 取得した辞書をd_listに格納する
        d_list.append(d)




0
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=1
30
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=2
60
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=3
90
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=4
120
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=5
150
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=6
180
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=7
210
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=8
240
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=9
270
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=10
300
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=11
330
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=12
360
https://www.homes.co.jp/chintai/kodate/wakayama/list/?page=13


In [2]:
df = pd.DataFrame(d_list)
df

,name,place,station1,station2,age,floor,rent,madori,menseki,feature
0,JR紀勢本線 黒江駅 徒歩21分,和歌山県海南市岡田,JR紀勢本線 黒江駅 徒歩21分,JR紀勢本線 海南駅 3.7km,新築,2階建,8.2万円,2LDK,72.8m²,広々としたリビングにキッチン・お洒落な室内が魅力的♪
1,仮）シアラ四箇郷北IVＢ号,和歌山県和歌山市有本,JR阪和線 紀伊中ノ島駅 徒歩22分,JR阪和線 六十谷駅 徒歩26分,新築,2階建,8万円,3LDK,74.52m²,ペット相談可（犬、猫）の戸建賃貸物件です。 敷地内2台駐車可。
2,仮）シアラ四箇郷北IVＡ号,和歌山県和歌山市有本,JR阪和線 紀伊中ノ島駅 徒歩22分,JR阪和線 六十谷駅 徒歩26分,新築,2階建,7.9万円,3LDK,74.52m²,ペット相談可（犬、猫）の戸建賃貸物件です。 敷地内2台駐車可。
3,La Ferio三葛12,和歌山県和歌山市三葛,JR紀勢本線 紀三井寺駅 徒歩14分,JR紀勢本線 宮前駅 徒歩33分,新築,2階建,8.8万円,2LDK,70.18m²,広々としたリビングにキッチン・お洒落な室内が魅力的♪
4,ヤマイチテラス西浜,和歌山県和歌山市西浜1290-8,南海和歌山港線 和歌山港駅 徒歩37分,JR紀勢本線 紀三井寺駅 4.6km,新築,2階建,9.5万円,3LDK,77.83m²,None
...,...,...,...,...,...,...,...,...,...,...
358,一戸建15669,和歌山県和歌山市木ノ本,南海加太線 八幡前駅 徒歩19分,南海加太線 中松江駅 徒歩28分,81年,1階建,4万円,5DK,79.2m²,敷金・礼金0円☆専用庭付き☆人気の平屋一戸建賃貸♪
359,松江西戸建,和歌山県和歌山市松江西3丁目,南海加太線 八幡前駅 徒歩6分,南海加太線 中松江駅 徒歩12分,103年,1階建,2.9万円,5SDK,140.49m²,お客様のお部屋探しを全力でサポートします。
360,一戸建12467,和歌山県和歌山市松江西3丁目,南海加太線 八幡前駅 徒歩7分,南海加太線 中松江駅 徒歩13分,108年,1階建,2.5万円,6DK,120m²,縁側付き♪大家族におススメの6DKの平屋タイプの一戸建て賃貸です！
361,JR和歌山線 岩出駅 徒歩16分,和歌山県岩出市備前40-2,JR和歌山線 岩出駅 徒歩16分,JR和歌山線 船戸駅 徒歩18分,43年,2階建,33万円,5DK,148.99m²,スタッフ一同、お客様のご来店をこころよりお待ちしております
